### Import packages 

In [1]:
import sqlite3
import pandas as pd

### Connect to database

In [2]:
conn = sqlite3.connect('amazon.db')
c = conn.cursor()

In [3]:
c.execute("""SELECT * FROM books
LIMIT 5; """) 

In [4]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,,books,num_copies
0,0,tfLVzUKOLVzerlXUjYNQ,4226324
1,1,WWqhABnhgBzraupSvRhD,4048899
2,2,SgGhHtQZSpiCJZhYwMTs,774329
3,3,mwSsIVfIpkgMFCrslwgJ,1359606
4,4,aPmjxOFWWqUEXhjPHcei,4703380


In [9]:
c.execute("""SELECT * FROM author
LIMIT 5; """) 

In [10]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,,author,book
0,0,wohsXgOEtw,tfLVzUKOLVzerlXUjYNQ
1,1,yqsBLClBZE,WWqhABnhgBzraupSvRhD
2,2,nBvQnlxYUP,SgGhHtQZSpiCJZhYwMTs
3,3,IEWhjYiLmA,mwSsIVfIpkgMFCrslwgJ
4,4,vTWnWXJBUu,aPmjxOFWWqUEXhjPHcei


### Questions

1. How many rows are in the author and books tables?

2. How many authors (share) didn't sell some of their book? 

3. How many duplicates are there in the author table?   

4. How many authors (share) didn't sell any book?

5. Return the names of these authors.

6. How many books in total each author has sold? 

7. Choose only authors who have sold more than 400K copies
    - use subquery first
    - can you use something else? 

8. Get all authors that have their name starting with 'a'
    - Use substr first
    - Can you use a wild card? 

9. Count the number of authors that have their name starting with 'a' and that have single books that
    sold more than 200K copis
    - Can you use WITH